In [77]:
# Dependencies
# ----------------------------------
from sqlalchemy import create_engine, Table
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.ext.automap import automap_base

from sqlalchemy import Column, Integer, String, Float, DateTime, Date, Time, func

In [78]:
import datetime as dt

In [79]:
# Sets an object to utilize the default declarative base in SQL Alchemy
Base = declarative_base()

In [80]:
# Creates Classes which will serve as the anchor points for our Table
class Update(Base):
    __tablename__ = 'database_update'
    id = Column(Integer, primary_key=True)
    update_time = Column(Time)
    update_date = Column(Date)
    update_datetime = Column(DateTime)
    update_type = Column(String(255))
    candidate_id_str = Column(String(255))

In [81]:
# Create Database Connection
engine = create_engine("sqlite:///../db/twitter_db.sqlite")
conn = engine.connect()

In [82]:
# Create a "Metadata" Layer That Abstracts our SQL Database
# ----------------------------------
# Create (if not already in existence) the tables associated with our classes.
Base.metadata.create_all(engine)

In [ ]:
# help(Base.metadata)

In [ ]:
Base.metadata.reflect(engine)

In [ ]:
Base.metadata.clear()

In [ ]:
for table in Base.metadata.tables:
    print(table)

In [73]:
Base.metadata.tables

immutabledict({'database_update': Table('database_update', MetaData(bind=None), Column('id', Integer(), table=<database_update>, primary_key=True, nullable=False), Column('update_time', Time(), table=<database_update>), Column('update_date', Date(), table=<database_update>), Column('update_datetime', DateTime(), table=<database_update>), Column('update_type', String(length=255), table=<database_update>), Column('candidate_id_str', String(length=255), table=<database_update>), schema=None)})

In [ ]:
# Base.metadata.remove(table = Update())

In [ ]:
# help(Table)

In [ ]:
# Base_AutoMap.metadata.remove(table = Update)

In [ ]:
# Use this to clear out the db
# ----------------------------------
# Base.metadata.drop_all(engine)

In [74]:
# Base.metadata.drop_all(engine)

In [14]:
# engine.execute("select * from database_update").fetchall()

In [49]:
from sqlalchemy import inspect

In [83]:
# Create the inspector and connect it to the engine
inspector = inspect(engine)

In [84]:
# Collect the names of tables within the database
inspector.get_table_names()

['database_update', 'moving_averages', 'tweet_data']

In [ ]:
# Using the inspector to print the column names within the 'tweet_data' table and its types
columns = inspector.get_columns('database_update')
for column in columns:
    print(column["name"], column["type"])

## Testing

In [ ]:
date_1 = dt.date(2019, 7, 30)
date_2 = dt.date(2019, 7, 15)
date_3 = dt.date(2019, 6, 30)

In [26]:
from sqlalchemy.orm import Session

In [27]:
session = Session(engine)

In [28]:
Base_automap = automap_base()

In [29]:
Base_automap.prepare(engine, reflect=True)

In [30]:
Tweets = Base_automap.classes.tweet_data

In [31]:
Moving_Averages = Base_automap.classes.moving_averages

In [32]:
Update = Base_automap.classes.database_update

In [ ]:
candidate_user_id = "939091"

In [ ]:
days_back = dt.timedelta(days=10)


In [ ]:
most_recent_date = session.query(func.max(Tweets.created_at_date)).filter(Tweets.user_id_str == candidate_user_id).first()
most_recent_date_object = most_recent_date[0]

In [ ]:
oldest_date = session.query(func.min(Tweets.created_at_date)).filter(Tweets.user_id_str == candidate_user_id).first()
oldest_date_object = oldest_date[0] + dt.timedelta(days = 10)

In [ ]:
update_query_all = session.query(Tweets.user_name,
    func.avg(Tweets.retweet_count),
    func.avg(Tweets.favorite_count)).\
    filter(Tweets.user_id == "939091").\
    filter(Tweets.created_at_date >= (oldest_date_object - days_back)).\
    filter(Tweets.created_at_date <= most_recent_date_object)
update_query_all

In [97]:
none_query = session.query(Moving_Averages, func.avg(Moving_Averages.retweet_moving_average)).filter(Moving_Averages.date < dt.date(2000, 9, 11)).first()

In [93]:
none_query = session.query(Moving_Averages).filter(Moving_Averages.date < dt.date(2000, 9, 11)).first()

In [61]:
none_query is None

False

In [101]:
if none_query[0], none_query[1], none_query[2] is None
    print(none_query[0])
else:
    print("not none")

SyntaxError: invalid syntax (<ipython-input-101-524e90d6d2dd>, line 1)

In [96]:
none_query is None


True

In [106]:
engine.execute("select * from database_update").fetchall()

[(1, '06:09:04.510413', '2019-08-03', '2019-08-03 06:09:04.510413', 'moving_averages - full', '939091'),
 (2, '06:11:03.264951', '2019-08-03', '2019-08-03 06:11:03.264951', 'moving_averages - full', '15808765'),
 (3, '06:11:31.208846', '2019-08-03', '2019-08-03 06:11:31.208846', 'moving_averages - full', '226222147'),
 (4, '06:11:59.724442', '2019-08-03', '2019-08-03 06:11:59.724442', 'moving_averages - full', '19682187'),
 (5, '06:12:30.209117', '2019-08-03', '2019-08-03 06:12:30.209117', 'moving_averages - full', '426028646'),
 (6, '06:13:12.525658', '2019-08-03', '2019-08-03 06:13:12.525658', 'moving_averages - full', '26637348'),
 (7, '06:13:47.276586', '2019-08-03', '2019-08-03 06:13:47.276586', 'moving_averages - full', '72198806'),
 (8, '06:14:07.438099', '2019-08-03', '2019-08-03 06:14:07.438099', 'moving_averages - full', '14709326'),
 (9, '06:14:23.613676', '2019-08-03', '2019-08-03 06:14:23.613676', 'moving_averages - full', '30354991'),
 (10, '06:14:51.914096', '2019-08-03'

In [ ]:
days_diff = (most_recent_date_object - oldest_date_object + dt.timedelta(days = 1)).days

In [ ]:
for days in range(0, days_diff):

    current_date = oldest_date_object + dt.timedelta(days = days)
    print(current_date)

    current_date_query = update_query_all.\
        filter(Tweets.created_at_date > (current_date - days_back)).\
        filter(Tweets.created_at_date <= current_date).first()
    
    print(current_date_query)

    candidate_name = current_date_query[0]
    retweet_moving_average = current_date_query[1]
    favorite_moving_average = current_date_query[2]

In [ ]:
session.query(Moving_Averages).filter(Moving_Averages.date == current_date).\
    filter(Moving_Averages.candidate_id_str == candidate_user_id).count()

In [ ]:
session.query(Moving_Averages.candidate_name, Moving_Averages.candidate_id_str, Moving_Averages.date).all()

In [ ]:
session.query(Update).delete()

In [ ]:
session.commit()

In [ ]:
session.query(Update).all()

In [ ]:
session.add(Update(update_date = date_1))

In [ ]:
session.new

In [ ]:
session.commit()

In [ ]:
session.query(Update.update_date).all()

In [ ]:
session.add(Update(update_date = date_2))

In [ ]:
session.add(Update(update_date = date_3))

In [ ]:
session.new

In [ ]:
session.commit()

In [ ]:
session.query(Update.update_date).all()

In [ ]:
query = session.query(func.min(Update.update_date)).first()
query

In [ ]:
type(query[0])

In [ ]:
query[0]

In [ ]:
help(session.query)

In [ ]:
date_4 = dt.date(2019, 8, 1)

In [ ]:
date_4 > query

In [ ]:
(date_4 - date_1).days

In [ ]:
session.add(Update_Record(updated_date = date_4))

In [ ]:
session.commit()

In [ ]:
session.query(Update.update_datetime).all()

In [ ]:
session.query(func.max(Update.update_date)).first()

In [ ]:
session.query(Moving_Averages.candidate_name, Moving_Averages.date, Moving_Averages.retweet_moving_average).all()

In [ ]:
session.query(Moving_Averages).filter(Moving_Averages.date == dt.date(2019, 7, 30)).\
            filter(Moving_Averages.candidate_id_str == "939091").delete()

In [ ]:
session.query(Moving_Averages).delete()

In [ ]:
session.query(Moving_Averages).count()

In [ ]:
session.query(Update).delete()

In [ ]:
session.commit()

In [ ]:
session.query(Moving_Averages).all()

In [ ]:
session.close()

In [25]:
session = Session(engine)

NameError: name 'Session' is not defined

In [ ]:
engine.execute("SELECT * FROM moving_averages").fetchall()

In [ ]:
today_datetime = dt.datetime.utcnow()
today_date = today_datetime.date()
thirty_days_ago = dt.date.today() - dt.timedelta(days = 30)

time_delta = today_date - thirty_days_ago + dt.timedelta(days = 1)
days_back = dt.timedelta(days = 10)

session = Session(engine)

moving_average_list = []

moving_average_query = session.query(Moving_Averages.candidate_name,
    Moving_Averages.date,
    Moving_Averages.retweet_moving_average, 
    Moving_Averages.favorite_moving_average).\
    filter(Moving_Averages.date > (thirty_days_ago - days_back)).\
    filter(Moving_Averages.date <= today_date)

In [ ]:
keys = ("user_name", "moving_average_date", "retweet_moving_average", "favorite_moving_average")
moving_average_list = []
for query in moving_average_query:
    list_query = list(query)
    list_query[1] = dt.datetime.strftime(list_query[1], "%Y-%m-%d")
    print(list_query[1])
    moving_average_dict = dict(zip(keys, list_query))
    moving_average_list.append(moving_average_dict)

In [ ]:
modified_list = []
for date in moving_average_list

In [ ]:
moving_average_query.count()

In [ ]:
moving_average_list